# MoReBo
MoReBo stands for Movie Recommendations Bot.

## Introduction

# Data input

In [21]:
import pandas as pd
from io import StringIO
import numpy as np

# Read the data
movies = pd.read_csv('data/movies.dat', sep='::', engine='python', names=['movieId', 'title', 'genres'], encoding="ISO-8859-1")
ratings = pd.read_csv('data/ratings.dat', sep='::', engine='python', names=['userId', 'movieId', 'rating', 'timestamp'])
users = pd.read_csv('data/users.dat', sep='::', engine='python', names=['userId', 'gender', 'age', 'occupation', 'zip-code'])

In [22]:
# Merge the data
merged = pd.merge(movies, ratings, on='movieId', how='inner')
merged = pd.merge(merged, users, on='userId', how='inner')

In [23]:
# Create a new column for the year
merged['year'] = merged['title'].str.extract('.*\((.*)\).*', expand=True)

# Remove the year from the title
merged['title'] = merged['title'].str.replace(r"\s\(\d{4}\)","")

C:\Users\fabio\AppData\Local\Temp\ipykernel_1612\4005153913.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  merged['title'] = merged['title'].str.replace(r"\s\(\d{4}\)","")


In [24]:
merged.to_csv('data/merged.csv', index=False)